# Instagram Scraper - Testing Notebook

Test the Instagram scraper implementation:
1. **InstagramScraper** - URL-based extraction (profiles, posts, reels, comments)
2. **InstagramSearchScraper** - Parameter-based discovery with `extra_params`


---

## Setup - Use Local Development Version

In [1]:
import os
import sys
from pathlib import Path

# Add local src to path (use development version, not installed)
project_root = Path.cwd().parent
src_path = project_root / "src"
if str(src_path) not in sys.path:
    sys.path.insert(0, str(src_path))

print(f"Using source from: {src_path}")

# Load environment variables
from dotenv import load_dotenv
load_dotenv(project_root / ".env")

# Get API token
API_TOKEN = os.getenv("BRIGHTDATA_API_TOKEN")
if not API_TOKEN:
    raise ValueError("BRIGHTDATA_API_TOKEN not found in environment")

print(f"API Token: {API_TOKEN[:10]}...{API_TOKEN[-4:]}")
print("Setup complete!")

Using source from: /Users/ns/Desktop/projects/sdk-python/src
API Token: 7011787d-2...3336
Setup complete!


## Import Instagram Scrapers

In [2]:
from brightdata import BrightDataClient

# Verify we're using local version
import brightdata
print(f"brightdata module location: {brightdata.__file__}")

# Initialize client
client = BrightDataClient(token=API_TOKEN)

# Verify Instagram scraper is accessible
print(f"\nInstagramScraper: {type(client.scrape.instagram).__name__}")
print(f"InstagramSearchScraper: {type(client.search.instagram).__name__}")

# Check for new methods (profiles discovery, reels_all)
print("\nSearch scraper methods:")
print([m for m in dir(client.search.instagram) if not m.startswith('_') and callable(getattr(client.search.instagram, m))])

brightdata module location: /Users/ns/Desktop/projects/sdk-python/src/brightdata/__init__.py

InstagramScraper: InstagramScraper
InstagramSearchScraper: InstagramSearchScraper

Search scraper methods:
['posts', 'posts_sync', 'profiles', 'profiles_sync', 'reels', 'reels_all', 'reels_all_sync', 'reels_sync']


---
# Part 1: InstagramScraper (URL-based Extraction)

Test URL-based extraction methods using `await` (required in Jupyter).

## 1.1 Profiles - Extract profile by URL

In [ ]:
# Test post extraction by URL
POST_URL = "https://www.instagram.com/p/C0xFHGRItPt/"

print(f"Scraping post: {POST_URL}")
print("This may take 1-3 minutes...\n")

async with client.scrape.instagram.engine:
    result = await client.scrape.instagram.posts(url=POST_URL, timeout=180)

print(f"Success: {result.success}")
print(f"Status: {result.status}")
print(f"Snapshot ID: {result.snapshot_id}")
print(f"Cost: ${result.cost:.4f}" if result.cost else "Cost: N/A")

if result.success and result.data:
    print("\n--- Post Data ---")
    data = result.data
    # Use actual API field names
    print(f"Post ID: {data.get('post_id', 'N/A')}")
    print(f"Shortcode: {data.get('shortcode', 'N/A')}")
    print(f"User: {data.get('user_posted', 'N/A')}")
    description = str(data.get('description', 'N/A') or 'N/A')
    print(f"Description: {description[:100]}...")
    print(f"Likes: {data.get('likes', 'N/A')}")
    print(f"Comments: {data.get('num_comments', 'N/A')}")
    print(f"Posted at: {data.get('date_posted', 'N/A')}")
    print(f"Content Type: {data.get('content_type', 'N/A')}")
else:
    print(f"\nError: {result.error}")

## 1.2 Posts - Extract post by URL

In [7]:
# Export raw data to JSON file for inspection
import json
from pathlib import Path

output_file = Path.cwd() / "instagram_post_result.json"

export_data = {
    "success": result.success,
    "status": result.status,
    "snapshot_id": result.snapshot_id,
    "cost": result.cost,
    "row_count": result.row_count,
    "data": result.data,
    "error": result.error,
}

with open(output_file, "w") as f:
    json.dump(export_data, f, indent=2, default=str)

print(f"Exported to: {output_file}")
print(f"\nData type: {type(result.data)}")
print(f"Data preview: {str(result.data)[:500]}...")

Exported to: /Users/ns/Desktop/projects/sdk-python/notebooks/instagram_post_result.json

Data type: <class 'dict'>
Data preview: {'url': 'https://www.instagram.com/p/DTGAZJQkg5k/', 'user_posted': 'harrypotter', 'description': 'time for us to embrace the inevitable 😔', 'num_comments': 60, 'date_posted': '2026-01-04T15:30:16.000Z', 'likes': 40470, 'photos': ['https://scontent-bos5-1.cdninstagram.com/v/t51.2885-15/610176878_18558585163030699_8176807390950763652_n.jpg?stp=dst-jpg_e15_fr_p1080x1080_tt6&_nc_ht=scontent-bos5-1.cdninstagram.com&_nc_cat=107&_nc_oc=Q6cZ2QEkh4dVLXfb9wB7SW8tba5SXhPxPoYVePY5t47wEhrB5SC7fSnvYIs9cFBvJET...


In [8]:
# Debug: See actual data structure
print("=== Raw Data ===")
print(f"Type: {type(result.data)}")
print(f"Data: {result.data}")

=== Raw Data ===
Type: <class 'dict'>
Data: {'url': 'https://www.instagram.com/p/DTGAZJQkg5k/', 'user_posted': 'harrypotter', 'description': 'time for us to embrace the inevitable 😔', 'num_comments': 60, 'date_posted': '2026-01-04T15:30:16.000Z', 'likes': 40470, 'photos': ['https://scontent-bos5-1.cdninstagram.com/v/t51.2885-15/610176878_18558585163030699_8176807390950763652_n.jpg?stp=dst-jpg_e15_fr_p1080x1080_tt6&_nc_ht=scontent-bos5-1.cdninstagram.com&_nc_cat=107&_nc_oc=Q6cZ2QEkh4dVLXfb9wB7SW8tba5SXhPxPoYVePY5t47wEhrB5SC7fSnvYIs9cFBvJETQLtA&_nc_ohc=m4clcQGltTAQ7kNvwGtd33G&_nc_gid=d0unG-mnNeH_KJhc3ChoZQ&edm=ANTKIIoBAAAA&ccb=7-5&oh=00_Afqd-Dygr6WTcuJGtH7X5axfpWO_O_XJAXRk6HBg7Kb-VA&oe=696E87CF&_nc_sid=d885a2'], 'latest_comments': [{'comments': '😂😂😂', 'user_commenting': 'chees_yrat', 'likes': 0, 'profile_picture': 'https://scontent-bos5-1.cdninstagram.com/v/t51.2885-19/612565466_17874854847470299_5203377400103807929_n.jpg?stp=dst-jpg_s150x150_tt6&efg=eyJ2ZW5jb2RlX3RhZyI6InByb2ZpbGVfcGljL

In [9]:
post_img_link_with_carausel ="https://www.instagram.com/p/DTLo9uhDPCn/"
post_img_link="https://www.instagram.com/p/DTGAZJQkg5k/"
reel_link="https://www.instagram.com/reel/DTQygzxD6QC/"


In [10]:
# Test post extraction by URL

POST_URL = post_img_link

print(f"Scraping post: {POST_URL}")
print("This may take 1-3 minutes...\n")

async with client.scrape.instagram.engine:
    result = await client.scrape.instagram.posts(url=POST_URL, timeout=180)

print(f"Success: {result.success}")
print(f"Status: {result.status}")
print(f"Snapshot ID: {result.snapshot_id}")
print(f"Cost: ${result.cost:.4f}" if result.cost else "Cost: N/A")

if result.success and result.data:
    print("\n--- Post Data ---")
    data = result.data
    # Correct API field names
    print(f"Post ID: {data.get('post_id', 'N/A')}")
    print(f"Shortcode: {data.get('shortcode', 'N/A')}")
    print(f"User: {data.get('user_posted', 'N/A')}")
    description = str(data.get('description', 'N/A') or 'N/A')
    print(f"Description: {description[:100]}...")
    print(f"Likes: {data.get('likes', 'N/A')}")
    print(f"Comments: {data.get('num_comments', 'N/A')}")
    print(f"Posted at: {data.get('date_posted', 'N/A')}")
    print(f"Content Type: {data.get('content_type', 'N/A')}")
else:
    print(f"\nError: {result.error}")

Scraping post: https://www.instagram.com/p/DTGAZJQkg5k/
This may take 1-3 minutes...

Success: True
Status: ready
Snapshot ID: sd_mkfabhyt24w4dmf365
Cost: $0.0020

--- Post Data ---
Post ID: 3802728663289564772
Shortcode: DTGAZJQkg5k
User: harrypotter
Description: time for us to embrace the inevitable 😔...
Likes: 40471
Comments: 60
Posted at: 2026-01-04T15:30:16.000Z
Content Type: Image


## 1.3 Reels - Extract reel by URL

In [12]:
# Test reel extraction by URL
REEL_URL = reel_link

print(f"Scraping reel: {REEL_URL}")
print("This may take 1-3 minutes...\n")

async with client.scrape.instagram.engine:
    result = await client.scrape.instagram.reels(url=REEL_URL, timeout=180)

print(f"Success: {result.success}")
print(f"Status: {result.status}")
print(f"Snapshot ID: {result.snapshot_id}")
print(f"Cost: ${result.cost:.4f}" if result.cost else "Cost: N/A")

if result.success and result.data:
    print("\n--- Reel Data ---")
    data = result.data
    # Show available keys first
    print(f"Available keys: {list(data.keys()) if isinstance(data, dict) else 'N/A'}")
    # Correct API field names for reels
    print(f"\nPost ID: {data.get('post_id', 'N/A')}")
    print(f"Shortcode: {data.get('shortcode', 'N/A')}")
    print(f"User: {data.get('user_posted', 'N/A')}")
    description = str(data.get('description', 'N/A') or 'N/A')
    print(f"Description: {description[:100]}...")
    print(f"Likes: {data.get('likes', 'N/A')}")
    print(f"Comments: {data.get('num_comments', 'N/A')}")
    print(f"Views: {data.get('views', 'N/A')}")
    print(f"Play Count: {data.get('video_play_count', 'N/A')}")
    print(f"Length: {data.get('length', 'N/A')} seconds")
    print(f"Product Type: {data.get('product_type', 'N/A')}")
    print(f"Video URL: {data.get('video_url', 'N/A')[:80]}..." if data.get('video_url') else "Video URL: N/A")
else:
    print(f"\nError: {result.error}")

Scraping reel: https://www.instagram.com/reel/DTQygzxD6QC/
This may take 1-3 minutes...

Success: True
Status: ready
Snapshot ID: sd_mkfag4wn1g4ijaer1b
Cost: $0.0020

--- Reel Data ---
Available keys: ['url', 'user_posted', 'description', 'hashtags', 'num_comments', 'date_posted', 'likes', 'views', 'video_play_count', 'top_comments', 'post_id', 'thumbnail', 'shortcode', 'content_id', 'product_type', 'coauthor_producers', 'tagged_users', 'length', 'video_url', 'audio_url', 'posts_count', 'followers', 'following', 'user_profile_url', 'is_paid_partnership', 'partnership_details', 'is_verified', 'profile_image_link', 'timestamp', 'input']

Post ID: 3805763842060821506_1315934698
Shortcode: DTQygzxD6QC
User: harrypotter
Description: physically, we're at our desks, mentally, we're sitting in front of a cosy fire in our knitted jumpe...
Likes: 41422
Comments: 156
Views: 314279
Play Count: 1079845
Length: 9.515 seconds
Product Type: clips
Video URL: https://scontent-lax3-2.cdninstagram.com/o1/

In [13]:
# Quick video download
import aiohttp
from pathlib import Path

async def download_video(url: str, filename: str) -> str:
    """Download video from URL."""
    output_path = Path.cwd() / filename
    async with aiohttp.ClientSession() as session:
        async with session.get(url) as response:
            if response.status == 200:
                with open(output_path, 'wb') as f:
                    f.write(await response.read())
                return str(output_path)
            else:
                return f"Error: {response.status}"

# Download the reel video
if result.success and result.data and result.data.get('video_url'):
    video_url = result.data['video_url']
    shortcode = result.data.get('shortcode', 'reel')
    filename = f"instagram_reel_{shortcode}.mp4"
    
    saved_path = await download_video(video_url, filename)
    print(f"Video saved to: {saved_path}")
else:
    print("No video URL available")

Video saved to: /Users/ns/Desktop/projects/sdk-python/notebooks/instagram_reel_DTQygzxD6QC.mp4


## 1.4 Comments - Extract comments by post URL

In [18]:
# Test comments extraction by URL
COMMENTS_URL = post_img_link  # Use the post we already tested

print(f"Scraping comments from: {COMMENTS_URL}")
print("This may take 1-3 minutes...\n")

async with client.scrape.instagram.engine:
    result = await client.scrape.instagram.comments(url=COMMENTS_URL, timeout=180)

print(f"Success: {result.success}")
print(f"Status: {result.status}")
print(f"Snapshot ID: {result.snapshot_id}")
print(f"Cost: ${result.cost:.4f}" if result.cost else "Cost: N/A")

if result.success and result.data:
    print("\n--- Comments Data ---")
    data = result.data
    if isinstance(data, list):
        print(f"Number of comments: {len(data)}")
        if len(data) > 0:
            print(f"Available keys: {list(data[0].keys())}")
        for i, comment in enumerate(data[:5]):
            print(f"\nComment {i+1}:")
            # Correct API field names for comments endpoint
            print(f"  User: {comment.get('comment_user', 'N/A')}")
            print(f"  User URL: {comment.get('comment_user_url', 'N/A')}")
            text = str(comment.get('comment', 'N/A'))
            print(f"  Text: {text[:80]}...")
            print(f"  Likes: {comment.get('likes_number', 'N/A')}")
            print(f"  Replies: {comment.get('replies_number', 'N/A')}")
            print(f"  Date: {comment.get('comment_date', 'N/A')}")
    elif isinstance(data, dict):
        print(f"Available keys: {list(data.keys())}")
        print(f"Data: {data}")
    else:
        print(f"Data type: {type(data)}")
else:
    print("\nNo data returned. Debug info:")
    print(f"  result.data: {result.data}")
    print(f"  result.row_count: {result.row_count}")
    print(f"  result.error: {result.error}")

Scraping comments from: https://www.instagram.com/p/DTGAZJQkg5k/
This may take 1-3 minutes...

Success: True
Status: ready
Snapshot ID: sd_mkfb04rs2nlefcer20
Cost: $0.0300

--- Comments Data ---
Number of comments: 15
Available keys: ['url', 'comment_user', 'comment_user_url', 'comment_date', 'comment', 'likes_number', 'replies_number', 'post_url', 'post_user', 'comment_id', 'post_id', 'timestamp', 'input']

Comment 1:
  User: annisaameilani.a
  User URL: https://www.instagram.com/annisaameilani.a
  Text: 😂😂😂...
  Likes: 0
  Replies: 0
  Date: 2026-01-04T22:23:15.000Z

Comment 2:
  User: swaywarm
  User URL: https://www.instagram.com/swaywarm
  Text: 💔...
  Likes: 0
  Replies: 0
  Date: 2026-01-04T21:03:49.000Z

Comment 3:
  User: manue_arsenault_
  User URL: https://www.instagram.com/manue_arsenault_
  Text: 😂...
  Likes: 0
  Replies: 0
  Date: 2026-01-04T19:39:56.000Z

Comment 4:
  User: michileobw
  User URL: https://www.instagram.com/michileobw
  Text: ❤️❤️❤️...
  Likes: 0
  Replie

---
# Part 2: InstagramSearchScraper (Discovery with extra_params)

Test parameter-based discovery methods that use `extra_params` for:
- `type=discover_new`
- `discover_by=user_name|url|url_all_reels`

## 2.1 Profiles Discovery - by username

In [ ]:
# Test profile discovery by username
# Uses: extra_params={"type": "discover_new", "discover_by": "user_name"}
USERNAME = "nasa"

print(f"Discovering profile for username: {USERNAME}")
print("Using extra_params: type=discover_new, discover_by=user_name")
print("This may take 1-3 minutes...\n")

async with client.search.instagram.engine:
    result = await client.search.instagram.profiles(user_name=USERNAME, timeout=180)

print(f"Success: {result.success}")
print(f"Status: {result.status}")
print(f"Snapshot ID: {result.snapshot_id}")
print(f"Cost: ${result.cost:.4f}" if result.cost else "Cost: N/A")

if result.success and result.data:
    print("\n--- Discovered Profile ---")
    data = result.data
    # Show data structure first
    if isinstance(data, list) and len(data) > 0:
        data = data[0]
    
    print(f"Available keys: {list(data.keys()) if isinstance(data, dict) else 'N/A'}")
    
    # Correct API field names for profile discovery
    print(f"\nAccount: {data.get('account', 'N/A')}")
    print(f"Full Name: {data.get('full_name', 'N/A')}")
    print(f"Profile Name: {data.get('profile_name', 'N/A')}")
    print(f"ID: {data.get('id', 'N/A')}")
    print(f"Followers: {data.get('followers', 'N/A'):,}" if isinstance(data.get('followers'), int) else f"Followers: {data.get('followers', 'N/A')}")
    print(f"Following: {data.get('following', 'N/A')}")
    print(f"Posts: {data.get('posts_count', 'N/A')}")
    print(f"Is Verified: {data.get('is_verified', 'N/A')}")
    print(f"Is Business: {data.get('is_business_account', 'N/A')}")
    print(f"Category: {data.get('category_name', 'N/A')}")
    bio = str(data.get('biography', 'N/A') or 'N/A')
    print(f"Bio: {bio[:100]}...")
    print(f"Profile URL: {data.get('profile_url', 'N/A')}")
    print(f"External URL: {data.get('external_url', 'N/A')}")
else:
    print(f"\nError: {result.error}")

## 2.2 Posts Discovery - by profile URL with filters

In [20]:
# Test posts discovery from profile URL
# Uses: extra_params={"type": "discover_new", "discover_by": "url"}
PROFILE_URL = "https://www.instagram.com/nasa/"

print(f"Discovering posts from profile: {PROFILE_URL}")
print("Using extra_params: type=discover_new, discover_by=url")
print("Filters: num_of_posts=5")
print("This may take 1-3 minutes...\n")

async with client.search.instagram.engine:
    result = await client.search.instagram.posts(
        url=PROFILE_URL,
        num_of_posts=5,
        timeout=180
    )

print(f"Success: {result.success}")
print(f"Status: {result.status}")
print(f"Snapshot ID: {result.snapshot_id}")
print(f"Cost: ${result.cost:.4f}" if result.cost else "Cost: N/A")

if result.success and result.data:
    print("\n--- Discovered Posts ---")
    data = result.data
    if isinstance(data, list):
        print(f"Number of posts discovered: {len(data)}")
        if len(data) > 0:
            print(f"Available keys: {list(data[0].keys()) if isinstance(data[0], dict) else 'N/A'}")
        for i, post in enumerate(data[:3]):
            print(f"\nPost {i+1}:")
            print(f"  URL: {post.get('url', 'N/A')}")
            print(f"  Post ID: {post.get('post_id', 'N/A')}")
            description = str(post.get('description', 'N/A') or 'N/A')
            print(f"  Description: {description[:60]}...")
            print(f"  Likes: {post.get('likes', 'N/A')}")
            print(f"  Comments: {post.get('num_comments', 'N/A')}")
    elif isinstance(data, dict):
        print(f"Available keys: {list(data.keys())}")
        print(f"Post ID: {data.get('post_id', 'N/A')}")
        description = str(data.get('description', 'N/A') or 'N/A')
        print(f"Description: {description[:60]}...")
    else:
        print(f"Data type: {type(data)}")
else:
    print(f"\nError: {result.error}")

Discovering posts from profile: https://www.instagram.com/nasa/
Using extra_params: type=discover_new, discover_by=url
Filters: num_of_posts=5
This may take 1-3 minutes...

Success: True
Status: ready
Snapshot ID: sd_mkfb7xwn29nxxarxe
Cost: $0.0100

--- Discovered Posts ---
Number of posts discovered: 5
Available keys: ['url', 'user_posted', 'description', 'hashtags', 'num_comments', 'date_posted', 'likes', 'photos', 'latest_comments', 'post_id', 'discovery_input', 'shortcode', 'content_type', 'pk', 'content_id', 'thumbnail', 'followers', 'posts_count', 'profile_image_link', 'is_verified', 'is_paid_partnership', 'partnership_details', 'user_posted_id', 'post_content', 'audio', 'profile_url', 'videos_duration', 'images', 'alt_text', 'photos_number', 'timestamp', 'input']

Post 1:
  URL: https://www.instagram.com/p/DTTInqQliCG
  Post ID: 3806424019865313414
  Description: Soaring through stormy Sagittarius seas

The Lagoon Nebula, ...
  Likes: 398349
  Comments: 1888

Post 2:
  URL: http

## 2.3 Reels Discovery - by profile URL

In [21]:
# Test reels discovery from profile URL
# Uses: extra_params={"type": "discover_new", "discover_by": "url"}
PROFILE_URL = "https://www.instagram.com/espn/"

print(f"Discovering reels from profile: {PROFILE_URL}")
print("Using extra_params: type=discover_new, discover_by=url")
print("Filters: num_of_posts=5")
print("This may take 1-3 minutes...\n")

async with client.search.instagram.engine:
    result = await client.search.instagram.reels(
        url=PROFILE_URL,
        num_of_posts=5,
        timeout=180
    )

print(f"Success: {result.success}")
print(f"Status: {result.status}")
print(f"Snapshot ID: {result.snapshot_id}")
print(f"Cost: ${result.cost:.4f}" if result.cost else "Cost: N/A")

if result.success and result.data:
    print("\n--- Discovered Reels ---")
    data = result.data
    if isinstance(data, list):
        print(f"Number of reels discovered: {len(data)}")
        if len(data) > 0:
            print(f"Available keys: {list(data[0].keys()) if isinstance(data[0], dict) else 'N/A'}")
        for i, reel in enumerate(data[:3]):
            print(f"\nReel {i+1}:")
            print(f"  URL: {reel.get('url', 'N/A')}")
            print(f"  Post ID: {reel.get('post_id', 'N/A')}")
            print(f"  User: {reel.get('user_posted', 'N/A')}")
            print(f"  Likes: {reel.get('likes', 'N/A')}")
            print(f"  Comments: {reel.get('num_comments', 'N/A')}")
    elif isinstance(data, dict):
        print(f"Available keys: {list(data.keys())}")
    else:
        print(f"Data type: {type(data)}")
else:
    print(f"\nError: {result.error}")

Discovering reels from profile: https://www.instagram.com/espn/
Using extra_params: type=discover_new, discover_by=url
Filters: num_of_posts=5
This may take 1-3 minutes...

Success: True
Status: ready
Snapshot ID: sd_mkfb8dy72d6vsz89c6
Cost: $0.0100

--- Discovered Reels ---
Number of reels discovered: 5
Available keys: ['url', 'user_posted', 'description', 'hashtags', 'num_comments', 'date_posted', 'likes', 'views', 'video_play_count', 'top_comments', 'post_id', 'thumbnail', 'shortcode', 'content_id', 'product_type', 'coauthor_producers', 'tagged_users', 'length', 'video_url', 'audio_url', 'posts_count', 'followers', 'following', 'user_profile_url', 'is_paid_partnership', 'is_verified', 'profile_image_link', 'timestamp', 'input', 'discovery_input']

Reel 1:
  URL: https://www.instagram.com/p/DStw-S5Dgmt/
  Post ID: 3795905448067860909_505182045
  User: sportscenter
  Likes: 305570
  Comments: 1878

Reel 2:
  URL: https://www.instagram.com/p/DTgR7XfDOLy/
  Post ID: 3810124131212255986_

## 2.4 Reels All Discovery - by profile URL

In [22]:
# Test reels_all discovery from profile URL
# Uses: extra_params={"type": "discover_new", "discover_by": "url_all_reels"}
PROFILE_URL = "https://www.instagram.com/billieeilish/"

print(f"Discovering ALL reels from profile: {PROFILE_URL}")
print("Using extra_params: type=discover_new, discover_by=url_all_reels")
print("Filters: num_of_posts=10")
print("This may take 1-3 minutes...\n")

async with client.search.instagram.engine:
    result = await client.search.instagram.reels_all(
        url=PROFILE_URL,
        num_of_posts=10,
        timeout=180
    )

print(f"Success: {result.success}")
print(f"Status: {result.status}")
print(f"Snapshot ID: {result.snapshot_id}")
print(f"Cost: ${result.cost:.4f}" if result.cost else "Cost: N/A")

if result.success and result.data:
    print("\n--- Discovered All Reels ---")
    data = result.data
    if isinstance(data, list):
        print(f"Number of reels discovered: {len(data)}")
    else:
        print(f"Data type: {type(data)}")
else:
    print(f"\nError: {result.error}")

Discovering ALL reels from profile: https://www.instagram.com/billieeilish/
Using extra_params: type=discover_new, discover_by=url_all_reels
Filters: num_of_posts=10
This may take 1-3 minutes...

Success: False
Status: timeout
Snapshot ID: sd_mkfbpfvl12xas6bvz2
Cost: N/A

Error: Polling timeout after 180s


---
# Part 3: Verify Timing Metadata

In [ ]:
# Check timing metadata from last result
print("=== Timing Metadata ===")
print(f"trigger_sent_at: {result.trigger_sent_at}")
print(f"snapshot_id_received_at: {result.snapshot_id_received_at}")
print(f"snapshot_polled_at: {result.snapshot_polled_at}")
print(f"data_fetched_at: {result.data_fetched_at}")
print(f"\nrow_count: {result.row_count}")
print(f"cost: {result.cost}")

---
# Summary

## InstagramScraper (URL-based)
- `profiles(url)` - Extract profile by URL
- `posts(url)` - Extract post by URL
- `reels(url)` - Extract reel by URL
- `comments(url)` - Extract comments by post URL

## InstagramSearchScraper (Discovery with extra_params)
- `profiles(user_name)` - Discover by username (`discover_by=user_name`)
- `posts(url, ...)` - Discover posts with filters (`discover_by=url`)
- `reels(url, ...)` - Discover reels (`discover_by=url`)
- `reels_all(url, ...)` - Discover all reels (`discover_by=url_all_reels`)